# Fáza 1. Prieskumná analýza

## 1.1 Základný opis dát spolu s ich charakteristikami

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, HTML
from scipy.stats import pearsonr,spearmanr

SEED = 42
np.random.seed(SEED)

DATA_DIR = "./115"

# Načítanie dát

df_obs = pd.read_csv(os.path.join(DATA_DIR, "observation.csv"), sep='\t', encoding='utf-8-sig')
df_pat = pd.read_csv(os.path.join(DATA_DIR, "patient.csv"), sep='\t', encoding='utf-8-sig')
df_sta = pd.read_csv(os.path.join(DATA_DIR, "station.csv"), sep='\t', encoding='utf-8-sig')


In [ ]:
# --- A. Analýza štruktúr dát ---

# Funkcia na rýchly prehľad datasetu

def dataset_summary(df, name):
    print("Dataset:",name)
    print("Počet riadkov: ",df.shape[0])
    print("Počet stĺpcov: ",df.shape[1])
    display(df.head())
    display(df.info())


dataset_summary(df_obs, "observation.csv")
dataset_summary(df_pat, "patient.csv")
dataset_summary(df_sta, "station.csv")



 ### 1.1(A) Analýza  štruktúr  dát  ako  súbory  
(štruktúry  a  vzťahy,  počet,  typy,  ...),  záznamy (štruktúry, počet záznamov, počet atribútov, typy, ...)


Projekt obsahuje 3 základné súbory:

- **observation.csv** – obsahuje senzorové merania pacienta (napr. SpO₂, HR, RR, tlak, teplotu kože,a iné)
- **patient.csv** – uchováva informácie o pacientoch ako základné(meno, priezvisko,mail,...) tak aj pokročilé(krvná skupina,dátum narodenia,...)
  

- **station.csv** – obsahuje informácie o staniciach, kde boli merania vykonávané (napr.location, stationm, QoS).


Z hľadiska dátových typov sú všetky súbory v poriadku – numerické premenné majú typ `float64`
a textové typ `object`. Dátumové a časové údaje možno budú vyžadovať
konverziu na typ `datetime64` 




### 1.1(B). Analýza jednotlivých atribútov: pre zvolené významné atribúty (min 10) analyzujte ich  distribúcie  a  základné  deskriptívne  štatistiky  a  či  spĺňa  predpísané  podmienky  a rozsah meraných hodnôt.

In [ ]:
attributes = ['SpO₂', 'HR', 'PI', 'RR', 'EtCO₂', 'FiO₂', 'BP', 'Skin Temperature', 'Motion/Activity index', 'CO']

for atr in attributes:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    

    print("Atribút:", atr,"\n")
    display(df_obs[atr].describe())
    axes[0].set_title(f"{atr} - Histogram")
    axes[1].set_title(f"{atr} - Boxplot")
    sns.histplot(df_obs[atr].dropna(), kde=True, ax=axes[0])
    sns.boxplot(x=df_obs[atr], ax=axes[1])
    plt.show()


### 1.1(C) Párová analýza dát: Identifikujte vzťahy a závislostí medzi dvojicami atribútov. 


In [ ]:
#vsetky numericke atributy
#corr_matrix = df_obs.select_dtypes(include=[np.number]).copy().corr()


#len tie podstatnejšie
corr_matrix = df_obs[['SpO₂', 'HR', 'PI', 'RR', 'EtCO₂', 'FiO₂', 'BP','PRV','PVI', 'Skin Temperature', 'Motion/Activity index', 'CO','SV',]].corr()


plt.figure(figsize=(10,10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matica vzťahov medzi atribútmi')
plt.show()

### 1.1(D) Párová  analýza  dát:  Identifikujte  závislosti  medzi  predikovanou  premennou  a ostatnými premennými (potenciálnymi prediktormi).

In [ ]:


#ciselne stplce
num = df_obs.select_dtypes(include=[np.number]).copy()

# korelacia 
corr = num.corr(numeric_only=True)['oximetry'].drop('oximetry')
corr_abs = corr.abs().sort_values(ascending=False)



pv = []
for col in corr_abs.index[:10]: # len top 10
    x = num[col].dropna()
    y = num['oximetry'].loc[x.index]
    r, p = pearsonr(x, y)
    pv.append((col, r, p))
pv_df = pd.DataFrame(pv, columns=['atribúty','r(Pearson)','p-value'])

# 4) Vizualizácia
plt.figure(figsize=(10,6))
sns.barplot(x=corr_abs.index, y=corr_abs.values, order=corr_abs.index)
plt.xticks(rotation=90)
plt.title('Velkosť vzťahu s oximetry (Pearson r)')
plt.xlabel('Atribút')
plt.ylabel('korelačný koeficient')
plt.tight_layout()
plt.show()

pv_df


### 1.1(E) Dokumentácia zistení

- Pri prvotnej analýze datasetov sme sa zamerali na štruktúru datasetov a vzťahmi medzi jednotlivými atribútami

- dataset pozostáva z troch častí **observation**, **patient**, **station** 

**observion** obsahuje jednotlivé hodnoty pozorovania

**patient** obsahuje informácie o pacientoch ako meno, email, krvná skupina, id a rôzne iné informácie

**station** je dataset obsahujúci stanice kde prebiehali merania, ich názov, miesto, ...


Pri práci s datasetmi sme zistili, že neexistuje priame napojenie csv súborov a nedá sa s nimi pracovať na tejto úrovni. To znamená, že nevieme koho sú skúmané hodnoty. Napojenie si môžeme všimnúť medzi observation a station podľa hodnoty latitude a longitude.



## 1.2 Identifikácia problémov, integrácia a čistenie dát

### 1.2(A)  Identifikujte  aj prvotne riešte problémy v dátach
napr.: nevhodná štruktúra dát, duplicitné  záznamy,  ktoré  môžu  vznikať  po  určitých  dátových  transformáciach, nejednotné formáty, chýbajúce hodnoty,vychýlenéhodnoty. V dátach sa môžu nachádzať aj iné, tu nevymenované problémy, resp. menej problémov ako bolo uvedených.

In [ ]:
datasets = {
    "Observation": df_obs,
    "Patient": df_pat,
    "Station": df_sta
}


for name, df in datasets.items():
    print("\n\n Dataset: ",name)
    print("Počet riadkov: ",df.shape[0])
    print("Počet stĺpcov: ",df.shape[1])

    print("Chýbajúce hodnoty na stĺpec:")
    if df.isnull().sum().sum() == 0:
        print("Žiadne chýbajúce hodnoty")
    else:
        print(df.isnull().sum())
    print("typy: \n",df.dtypes)

    duplicitne_indexy = df[df.duplicated()].index
    if duplicitne_indexy.empty:
        print("Žiadne duplicitné riadky")
    else:
        print("Indexy duplicitných riadkov:", duplicitne_indexy.tolist())
        

    print("pocet unikatnych hodnot na stlpec:")
    print(df.nunique())









In [ ]:

#vypisduplicitneho riadku v observation
dups_obs = df_obs[df_obs.duplicated(keep=False)].sort_values(list(df_obs.columns))
display(dups_obs)


**Observation**
- Po dôkladnom výise sme zistili duplicitný záznam v datasete observation na riadku 3850. V nižšie uvedenom výpise je vidieť aj porovnanie s riadkom s ktorým duplicitu zdieľa. 
- Ostatné hodnoty vyzerajú byť v poriadku

**Patient** 
- Tu môžeme vidieť, že stĺpec residence je prázdny(počet riadkov = počet chýbajúcich hodnôt v stĺpci) a môžeme ho teda úplne odstrániť. 
- Taktiež vidíme veľký počet chýbajúcich záznamov v stĺpcoch ako **job**,**address** a **birthdate**. Dalšia chyba je v type stĺcov **registration** a **birthdate**, ktoré su *object* a maju byť *datetime*

**Station**
- v datasete station sme nezaznamenali podozrivé anomálie


#### Riešenia

In [ ]:
#zatial zakomentovane odstranenie duplicitnych riadkov
#v datasete observation.csv
#past = df_obs.shape[0]
#df_obs = df_obs.drop_duplicates()
#future = df_obs.shape[0]
#print(f"Observation: odstránených {past - future} duplicitných riadkov")


#odstranenie stplca NaN v residence.csv
#if 'residence' in df_patient.columns and df_patient['residence'].isna().all():
    #df_patient = df_patient.drop(columns=['residence'])


#zmena datovych typov na z float na datumy
# parsovanie dátumov
#for col in ['registration', 'birthdate']:
    #if col in df_patient.columns:
        #df_patient[col] = pd.to_datetime(df_patient[col], errors='coerce')




### 1.2(B)Kontrola správnosť v dátach 
-  či obsahujú abnormálne hodnoty 
-  či  obsahujú  nelogické  dátové  vzťahy,  ktoré  sú následkom  dátovej  kolekcie  a anotovania dát

In [ ]:
#abnormalne data

normal_ranges = {
    'SpO₂': (70, 100),
    'HR': (40, 180),
    'PI': (0.3, 20),
    'RR': (8, 30),
    'EtCO₂': (20, 50),
    'FiO₂': (21, 100),
    'PRV': (0, 300),
    'BP': (60, 180),
    'Skin Temperature': (30, 40),
    'Motion/Activity index': (0, 100),
    'PVI': (0, 30),
    'Hb level': (10, 18),
    'SV': (40, 200),
    'CO': (3, 8),
    'Signal Quality Index': (0, 100),
    'Respiratory effort': (0, 100),
    'O₂ extraction ratio': (0.1, 0.7),
    'SNR': (0, 80)
}



print("Počet abnormálnych hodnôt podľa medicínskych rozsahov:\n")
for col, (low, high) in normal_ranges.items():
    if col in df_obs.columns:
        mask = (df_obs[col] < low) | (df_obs[col] > high)
        count = mask.sum()
        print(f"{col:20s}: {count:4d} ({round(count/len(df_obs)*100,2)}%) mimo rozsah")


In [ ]:
# Nelogické kombinácie
#možno pridat ďalšie podľa potreby
rules = {
    "SpO₂=0 a HR>0": (df_obs['SpO₂'] == 0) & (df_obs['HR'] > 0),
    "CO=0 a HR>0": (df_obs['CO'] == 0) & (df_obs['HR'] > 0),
    "FiO₂=21 a EtCO₂=0": (df_obs['FiO₂'] == 21) & (df_obs['EtCO₂'] == 0)
}

print("Observattion: \n")
print("\nNelogické kombinácie hodnôt:")
for name, mask in rules.items():
    print(f"{name}: {mask.sum()} riadkov ({mask.mean()*100:.2f} %)")


#patient.csv
print("\nPatient: \n")

invalid_blood = ~df_pat['blood_group'].isin(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', '0+', '0-','O-','O+'])
print("Neplatné krvné skupiny:", invalid_blood.sum())


# 2. Duplicitné SSN
print("Duplicitné SSN:", df_pat['ssn'].duplicated().sum())






Nelogické kombinácie hodnôt:
SpO₂=0 a HR>0: 0 riadkov (0.00 %)
CO=0 a HR>0: 0 riadkov (0.00 %)
FiO₂=21 a EtCO₂=0: 0 riadkov (0.00 %)
Neplatné krvné skupiny: 0
Duplicitné SSN: 0


#### Záznam výsledkov
- zistili sme, že v datasetoch sa nenachádzajú nabnormálne hodnoty ani nelogické dátové vzťahy hlavne v datasete **observation.csv**
- na druhú stranu v datasete **patient.csv** sme zistili, že krvá skupina 0+ a 0- sú zapísané ako veľké písmeno O a nie číslica 0(zatiaľ ponechané tak ako bolo)